In [0]:
# Import the useful libraries 
import numpy as np
import pandas as pd
import os
from random import randint
import matplotlib.pyplot as plt
from google.colab import files
import time

In [2]:
# Define R Matrix 

R = np.matrix([[0, 0, np.nan, np.nan, 0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
                [0, 0, 0, np.nan, 0, 0, np.nan, np.nan, np.nan, np.nan, np.nan],
                [np.nan, 0, 0, 0, np.nan, np.nan, 100, np.nan, np.nan, np.nan, np.nan],
                [np.nan, np.nan, 0, 0, np.nan, np.nan, 100, np.nan, np.nan, np.nan, np.nan],
                [0, 0, np.nan, np.nan, 0, 0, np.nan, np.nan, 0, 0, np.nan],
                [np.nan, 0, np.nan, np.nan, 0, 0, 100, 0, np.nan, np.nan, np.nan],
                [np.nan, np.nan, 0, 0, np.nan, 0, 100, 0, np.nan, np.nan, np.nan],
                [np.nan, np.nan, np.nan, np.nan, np.nan, 0, 100, 0, np.nan, np.nan, 0],
                [np.nan, np.nan, np.nan, np.nan, 0, np.nan, np.nan, np.nan, 0, 0, np.nan],
                [np.nan, np.nan, np.nan, np.nan, 0, np.nan, np.nan, np.nan, 0, 0, 0],
                [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 0, np.nan, 0, 0]])

print(R)

[[  0.   0.  nan  nan   0.  nan  nan  nan  nan  nan  nan]
 [  0.   0.   0.  nan   0.   0.  nan  nan  nan  nan  nan]
 [ nan   0.   0.   0.  nan  nan 100.  nan  nan  nan  nan]
 [ nan  nan   0.   0.  nan  nan 100.  nan  nan  nan  nan]
 [  0.   0.  nan  nan   0.   0.  nan  nan   0.   0.  nan]
 [ nan   0.  nan  nan   0.   0. 100.   0.  nan  nan  nan]
 [ nan  nan   0.   0.  nan   0. 100.   0.  nan  nan  nan]
 [ nan  nan  nan  nan  nan   0. 100.   0.  nan  nan   0.]
 [ nan  nan  nan  nan   0.  nan  nan  nan   0.   0.  nan]
 [ nan  nan  nan  nan   0.  nan  nan  nan   0.   0.   0.]
 [ nan  nan  nan  nan  nan  nan  nan   0.  nan   0.   0.]]


In [3]:
# Initialise Q Matrix 
Q = np.matrix(np.zeros([11,11]))

print(Q)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [0]:
def q_learning(alpha, gamma, epsilon, decay):
    
  # initialise current state in random
  current_state = randint(0, 10)
  
  # check available actions in the current state
  current_state_row = R[current_state,]
  av_act = np.where(current_state_row >= 0)[1]
  
  # randomly select the next action and record in steps
  next_action = int(np.random.choice(av_act,1))
  
  # epislon greedy policy
  if np.random.uniform(0, 1) > epsilon:
    one_step_ahead_q_value = np.max(Q[next_action, :])
  else:
    one_step_ahead_q_value = np.random.choice(np.array(Q[next_action,:]).ravel())
    
  # update Q-matrix
  Q[current_state, next_action] = Q[current_state, next_action] + alpha * (R[current_state, next_action] + gamma * one_step_ahead_q_value - Q[current_state, next_action])
  
  # update epsilon value
  epsilon *= decay
  
  # return Q-matrix and number of steps
  return Q, alpha, gamma, epsilon, decay

In [0]:
def evaluate(Q, episode, alpha, gamma, epsilon, decay):
  
  # initiate current state
#   current_state = randint(0,10)
  current_state = 8
  steps = [current_state]
  reward = 0
  
  # set rule if initiated from Station 7
  if current_state == 6:
    reward += np.max(Q[current_state, :])
  
  # step for start from other stations
  else:
    while current_state != 6:
      
      # check available actions, particular for early episode that Q value is 0 
      current_state_row = R[current_state, :]
      av_act = np.where(current_state_row >= 0)[1]
      
      # index for maximum value
      max_index = np.where(Q[current_state,:] == np.max(Q[current_state, av_act]))[1]
      
      if max_index.shape[0] > 1:
        next_step = int(np.random.choice(max_index, size = 1))
      else:
        next_step = int(max_index)
      
      # append next action and reward
      next_reward = np.max(Q[current_state, max_index])
      
      steps.append(next_step)
      reward += next_reward
      current_state = next_step
      
  
  average_reward.append((episode, alpha, gamma, epsilon, decay, reward/len(steps)))  
  step_cont.append((episode, alpha, gamma, epsilon, decay, len(steps)))
  
  return average_reward, step_cont

In [0]:
# Define initial parameters 
alphas = [0.05, 0.1, 0.2, 0.5, 0.7, 0.9, 1] # alpha is the learning rate
gammas = [0.8] # gamma is the discount factor 
epsilons = [0.1] # epsilon is the exploration factor
decays = [0.95] # df is the decay factor for epsilon

In [7]:
for alpha in alphas:
    for gamma in gammas:
        for epsilon in epsilons:
            for decay in decays:
                print(alpha,gamma,epsilon,decay)

0.05 0.8 0.1 0.95
0.1 0.8 0.1 0.95
0.2 0.8 0.1 0.95
0.5 0.8 0.1 0.95
0.7 0.8 0.1 0.95
0.9 0.8 0.1 0.95
1 0.8 0.1 0.95


In [8]:
average_reward = []
step_cont = []

start = time.time()

for alpha in alphas:
  for gamma in gammas:
    for decay in decays:
      for epsilon in epsilons:
        for episode in range(100):

          Q, alpha, gamma, epsilon, decay = q_learning(alpha, gamma, epsilon, decay)
    
          if np.matrix.max(Q) > 0:
            Q_norm = (Q/np.matrix.max(Q)*100)
          else:
            Q_norm = Q

          average_reward, step_cont = evaluate(Q_norm, episode, alpha, gamma, epsilon, decay)
      
      # Reset the Q-matrix and epsilon to initial value
      Q = np.matrix(np.zeros([11,11]))
      epsilon = epsilon
          
end = time.time()
                  
print(Q_norm.astype(int))
# print(average_reward)
# print(step_cont)
print('Time used :', end-start, 'seconds')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater_equal
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater_equal


[[  0  24   0   0   0   0   0   0   0   0   0]
 [  0  24  30   0  19  24   0   0   0   0   0]
 [  0  24  49  39   0   0  62   0   0   0   0]
 [  0   0  49  80   0   0 100   0   0   0   0]
 [  0  24   0   0  19   0   0   0  15   0   0]
 [  0  24   0   0  19  24   0  30   0   0   0]
 [  0   0  49  30   0  24  77  30   0   0   0]
 [  0   0   0   0   0  24  37  30   0   0  24]
 [  0   0   0   0   0   0   0   0  15  19   0]
 [  0   0   0   0   0   0   0   0   0  19  24]
 [  0   0   0   0   0   0   0  30   0   0  24]]
Time used : 0.32828474044799805 seconds


In [9]:
df_reward = pd.DataFrame.from_records(average_reward, columns = ['episode', 'alpha', 'gamma', 'epsilon', 'decay', 'Average Reward Per Step'])
df_step = pd.DataFrame.from_records(step_cont, columns = ['episode', 'alpha', 'gamma', 'epsilon', 'decay','Steps'])

df = pd.merge(df_reward, df_step, on=['episode', 'alpha', 'gamma', 'epsilon', 'decay'])
print(df.shape)
df.head(5)

(700, 7)


,episode,alpha,gamma,epsilon,decay,Average Reward Per Step,Steps
0,0,0.05,0.8,0.095000,0.95,0.0,25
1,1,0.05,0.8,0.090250,0.95,0.0,13
2,2,0.05,0.8,0.085737,0.95,0.0,19
3,3,0.05,0.8,0.081451,0.95,0.0,6
4,4,0.05,0.8,0.077378,0.95,0.0,8


In [0]:
df.to_excel('grid_search.xlsx')
files.download('grid_search.xlsx')